### **SELENIUM CRAWLING** *(Coupang)*

---

#### **SET UP**

##### *install*

In [1]:
# !pip install pandas
# !pip install supabase
# !pip install selenium
# !pip install beautifulsoup4
# !pip install tqdm
# !pip install webdriver-manager

In [2]:
import platform
import re, requests, csv, sys, os, zipfile

os_name = platform.system().lower()
architecture = platform.machine()

import requests
from bs4 import BeautifulSoup
 
import pandas as pd
from datetime import datetime
import time
from time import sleep

##### *library*

In [3]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from dateutil import parser
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

##### *chrome driver*

In [8]:
chrome_driver_url = 'https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.60/win64/chromedriver-win64.zip'

In [9]:
from glob import glob

driver_path = None
if os_name == 'darwin': # 맥 사용자
    driver_path = glob('./driver/**/chromedriver', recursive=True)[0]
else: # 윈도우 사용자
    driver_path = glob('./driver/**/chromedriver.exe', recursive=True)[0]

driver_path

'./driver\\chromedriver-win64\\chromedriver.exe'

In [10]:
# 드라이버 파일의 권한을 확인하고 수정
if os_name == 'darwin':  # 맥 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가
else:  # 윈도우 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가


In [11]:
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
# 어뷰징 우회
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

#### **WORK SPACE**

In [12]:
url = 'https://www.coupang.com/vp/products/7353654645?itemId=18929044583&vendorItemId=88886808373&pickType=COU_PICK&q=%EB%93%9C%EC%8B%9C%EC%98%A8+%EA%B3%A4%EC%95%BD%EB%B0%A5&itemsCount=36&searchId=ab5e3a984f364f0e80f1b0f4112e7eb6&rank=1&isAddedCart='

##### *chrome setting*

In [13]:
# Web OPEN
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)

In [14]:
# Page Loading
for i in range(3):
    print(f"page down no :{i}")
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(2)

page down no :0
page down no :1
page down no :2


In [15]:
# Review Click
driver.find_element(By.XPATH,'//*[@id="btfTab"]/ul[1]/li[2]').click()

##### *Crawling*

In [17]:
# Parsing
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [18]:
# Title selector
selector_title = soup.select_one('title').text

In [19]:
# Count page
count_page = (soup.select_one('#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__average.js_reviewAverageContainer > section.sdp-review__average__total-star > div.sdp-review__average__total-star__info > div.sdp-review__average__total-star__info-count').text)
page_int = int(count_page.replace(',', ''))
page_no = (page_int + 5 - 1)//5
next_no = page_no // 10

In [14]:
# Page Selector
selector_page = soup.select_one('#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(3)')

In [58]:
# container selector
good = ('#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > article:nth-child(4)')
selector = soup.select_one(good)

In [59]:
# Id selector
id = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span').text

In [20]:
# Created selector
created = parser.parse(selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date').text)

In [31]:
# Rate selector
rate = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray')

In [37]:
# product_option
product_option = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info__name').text

In [39]:
# content selector
content = selector.select_one('div.sdp-review__article__list__review.js_reviewArticleContentContainer > div').text

In [22]:
data = []

In [23]:
for _ in range(next_no):
    # 각 페이지를 순회하며 리뷰 데이터를 추출
    for j in range(2, 13):
        # 페이지 버튼을 클릭하여 이동
        page_button_xpath = f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[{j}]'
        try:
            page_button = driver.find_element(By.XPATH, page_button_xpath)
            page_button.click()
            time.sleep(2)  # 페이지가 로드될 시간을 기다림
        except Exception as e:
            print(f'Error clicking page {j}: {e}')
            continue
        
        # 현재 페이지의 HTML을 가져옴
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 현재 페이지의 리뷰 데이터 추출
        for i in range(3, 8):
            selector_str = f'#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > article:nth-child({i})'
            selector = soup.select_one(selector_str)
            
            if selector:
                try:
                    # Id selector
                    user_id = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span').text.strip()
                    
                    # Created selector
                    created = parser.parse(selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date').text.strip())
                    
                    # Rate selector
                    rate_element = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray')
                    rate = rate_element.get('data-rating') if rate_element else None
                    
                    # Product option selector
                    product_option = selector.select_one('div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info__name').text.strip()
                    
                    # Content selector
                    content = selector.select_one('div.sdp-review__article__list__review.js_reviewArticleContentContainer > div').text.strip()
                    
                    # 데이터 리스트에 추가
                    data.append({
                        'User ID': user_id,
                        'Created': created,
                        'Rate': rate,
                        'Product Option': product_option,
                        'Content': content
                    })
                except AttributeError as e:
                    print(f'Error processing review {i}: {e}')
            else:
                print(f'Review {i} not found on page {j}')

    # 다음 섹션으로 이동
    next_section_xpath = '//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[12]'
    try:
        next_section_button = driver.find_element(By.XPATH, next_section_xpath)
        next_section_button.click()
        time.sleep(2)  # 섹션이 로드될 시간을 기다림
    except Exception as e:
        print(f'Error clicking next section button: {e}')
        break

Error clicking page 5: Message: element click intercepted: Element <button class="sdp-review__article__page__num  js_reviewArticlePageBtn" data-page="...">4</button> is not clickable at point (459, 16). Other element would receive the click: <li name="review" class="active">...</li>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF664D822C2+60002]
	(No symbol) [0x00007FF664CFCA59]
	(No symbol) [0x00007FF664BB7EDA]
	(No symbol) [0x00007FF664C0F39E]
	(No symbol) [0x00007FF664C0CD62]
	(No symbol) [0x00007FF664C0A1FB]
	(No symbol) [0x00007FF664C093C6]
	(No symbol) [0x00007FF664BFB4D1]
	(No symbol) [0x00007FF664C2C28A]
	(No symbol) [0x00007FF664BFADF6]
	(No symbol) [0x00007FF664C2C4A0]
	(No symbol) [0x00007FF664C4BCC7]
	(No symbol) [0x00007FF664C2C033]
	(No symbol) [0x00007FF664BF9657]
	(No symbol) [0x00007FF664BFA251]
	GetHandleVerifier [0x00007FF665093E2D+3278285]
	GetHandleVerifier [0x00007FF6650E0190+3590448]
	GetHandleVerifier [0x00007FF6650D61D0+3549552

In [24]:
result_df = pd.DataFrame(data)

In [25]:
driver.close()
driver.quit()

In [26]:
result_df.to_csv('reviews.csv', index=False, encoding='utf-8-sig')